In [1]:
from __future__ import annotations

import os 

from torch.utils.data import random_split

from mmpfn.datasets.pad_ufes_20 import PADUFES20Dataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score
from mmpfn.models.mmpfn_v2 import MMPFNClassifier
from mmpfn.models.mmpfn_v2.constants import ModelInterfaceConfig
from mmpfn.models.mmpfn_v2.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune_mm.finetune_tabpfn_main import fine_tune_tabpfn

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
image_type='cls' # patch
# data_path = os.path.join(os.getenv('HOME'), "workspace/works/tabular_image/MultiModalPFN/mmpfn/data/pad_ufes_20")
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/pad_ufes_20")
dataset = PADUFES20Dataset(data_path)
_ = dataset.get_embeddings(type=image_type)

Load embeddings from embeddings/pad_ufes_20/pad_ufes_20_cls.pt


In [3]:
accuracy_scores = []
for seed in range(10):
    torch.manual_seed(seed)
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
    image_train = train_dataset.dataset.embeddings[train_dataset.indices]
    image_test = test_dataset.dataset.embeddings[test_dataset.indices]

    if image_type == 'cls':
        image_train = image_train.unsqueeze(1)
        image_test = image_test.unsqueeze(1)
        
    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_mmpfn_pad_ufes_20.ckpt"
    
    fine_tune_tabpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=None,
        image_train=image_train,
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="multiclass",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
        freeze_input=True,  # Freeze the input layers (encoder and y_encoder) during finetuning
        mixer_type='cross_merger' # no_append token_append split_append multihead cross_merger glu
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = MMPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
        mixer_type='cross_merger' # no_append token_append split_append multihead cross_merger glu
    )

    clf_finetuned = model_finetuned.fit(None, image_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(None, image_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-08-28 21:34:39,186] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   3%|▎         | 3/100 [00:01<01:25,  1.13it/s, Best Val. Loss=1.42, Best Val. Score=-1.42, Training Loss=1.41, Val. Loss=1.55, Patience=49, Utilization=0, Grad Norm=17] [2025-08-28 21:34:40,727] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   4%|▍         | 4/100 [00:02<01:10,  1.36it/s, Best Val. Loss=1.42, Best Val. Score=-1.42, Training Loss=1.47, Val. Loss=1.55, Patience=49, Utilization=0, Grad Norm=nan][2025-08-28 21:34:41,235] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   5%|▌         | 5/100 [00:02<01:01,  1.55it/s, Best Val. Loss=1.42, Best Val. Score=-1.42, Training Loss=1.41, Val. Loss=1.55, Patience=49, Utilization=0, Grad Norm=nan][2025-08-28 21:34:41,805] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning

accuracy_score (Finetuned): 0.6717391304347826


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-08-28 21:36:29,882] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   2%|▏         | 2/100 [00:02<04:31,  2.77s/it, Best Val. Loss=1.37, Best Val. Score=-1.37, Training Loss=1.44, Val. Loss=1.37, Patience=50, Utilization=0, Grad Norm=nan]


KeyboardInterrupt: 

In [ ]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.6369565217391304
Std Accuracy: 0.02184755569808889
